# HumanEval MBPP-Style Conversion Results Inspection

This notebook inspects results from `test_humaneval_mbpp_style.py` to verify:
- Whether MBPP-style format eliminates "comment-only" generation failures
- Compare original HumanEval vs MBPP-style prompts
- Analyze pass/fail distribution
- Inspect specific failing cases

In [42]:
import pandas as pd
import os
from pathlib import Path
import glob

# Set pandas display options to show FULL content
pd.set_option('display.max_colwidth', None)  # Show full column content (NO LIMITS)
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', None)         # Don't wrap to multiple lines
pd.set_option('display.max_rows', 50)        # Limit rows for readability

print("✅ Pandas display options set to show FULL content")

✅ Pandas display options set to show FULL content


In [43]:
# Auto-discovery of test results
datasets_dir = "../data/test_humaneval_mbpp_style/"
pattern = os.path.join(datasets_dir, "results_*.parquet")
matching_files = glob.glob(pattern)

if matching_files:
    matching_files.sort()
    print("=" * 80)
    print("HUMANEVAL MBPP-STYLE TEST RESULTS")
    print("=" * 80)
    print(f"\n🔍 Found {len(matching_files)} result file(s)")
    for file in matching_files:
        file_size = os.path.getsize(file) / 1024  # KB
        print(f"  📁 {Path(file).name} ({file_size:.2f} KB)")
else:
    raise FileNotFoundError(f"❌ No result files found in {datasets_dir}\n"
                           f"Run: python3 tests/test_humaneval_mbpp_style.py --n 20")

HUMANEVAL MBPP-STYLE TEST RESULTS

🔍 Found 1 result file(s)
  📁 results_gemma_2_2b.parquet (198.18 KB)


In [44]:
# Load the most recent results file
latest_file = matching_files[-1]  # Most recent (alphabetically)
print(f"\n📄 Loading: {latest_file}")

df = pd.read_parquet(latest_file)

print(f"\n" + "=" * 80)
print("BASIC STATISTICS")
print("=" * 80)
print(f"\n📊 Total records: {len(df)}")
print(f"📋 Columns: {list(df.columns)}")


📄 Loading: ../data/test_humaneval_mbpp_style/results_gemma_2_2b.parquet

BASIC STATISTICS

📊 Total records: 164
📋 Columns: ['task_id', 'task_idx', 'passed', 'is_comment_only', 'original_prompt', 'mbpp_style_prompt', 'generated_text', 'extracted_code', 'test_list', 'entry_point']


In [45]:
# Pass/Fail/Comment-Only Statistics
print("\n" + "=" * 80)
print("PASS/FAIL/COMMENT-ONLY STATISTICS")
print("=" * 80)

n_passed = df['passed'].sum()
n_comment_only = df['is_comment_only'].sum()
n_failed = len(df) - n_passed
n_failed_code = n_failed - n_comment_only  # Failed but generated actual code

print(f"\n✅ Passed:        {n_passed}/{len(df)} ({100*n_passed/len(df):.1f}%)")
print(f"❌ Failed:        {n_failed}/{len(df)} ({100*n_failed/len(df):.1f}%)")
print(f"   ├─ Code failures:    {n_failed_code} (generated code but tests failed)")
print(f"   └─ Comment-only:     {n_comment_only} (generated only comments, no code)")

if n_comment_only == 0:
    print(f"\n🎉 SUCCESS: No comment-only generations! MBPP-style format works!")
else:
    print(f"\n⚠️  Still have {n_comment_only} comment-only generations.")


PASS/FAIL/COMMENT-ONLY STATISTICS

✅ Passed:        31/164 (18.9%)
❌ Failed:        133/164 (81.1%)
   ├─ Code failures:    133 (generated code but tests failed)
   └─ Comment-only:     0 (generated only comments, no code)

🎉 SUCCESS: No comment-only generations! MBPP-style format works!


In [46]:
# Show comment-only failures (if any)
print("\n" + "=" * 80)
print("COMMENT-ONLY FAILURES")
print("=" * 80)

comment_only_df = df[df['is_comment_only'] == True]

if len(comment_only_df) == 0:
    print("\n✅ No comment-only failures!")
else:
    print(f"\n⚠️  {len(comment_only_df)} tasks generated only comments:")
    for idx, row in comment_only_df.iterrows():
        print(f"\n--- Task {row['task_idx']} ({row['task_id']}) ---")
        print(f"Extracted code:")
        print(row['extracted_code'][:500])
        if len(row['extracted_code']) > 500:
            print("...")


COMMENT-ONLY FAILURES

✅ No comment-only failures!


In [47]:
# Compare Original vs MBPP-Style Prompts for a sample task
print("\n" + "=" * 80)
print("PROMPT COMPARISON (Sample Tasks)")
print("=" * 80)

# Show first 3 tasks
for idx, row in df.head(3).iterrows():
    status = "PASS" if row['passed'] else ("COMMENT" if row['is_comment_only'] else "FAIL")
    
    print(f"\n{'=' * 80}")
    print(f"Task {row['task_idx']} ({row['task_id']}) [{status}]")
    print("=" * 80)
    
    print(f"\n📝 ORIGINAL HUMANEVAL PROMPT:")
    print("-" * 40)
    print(row['original_prompt'])
    
    print(f"\n📝 MBPP-STYLE PROMPT:")
    print("-" * 40)
    print(row['mbpp_style_prompt'])
    
    print(f"\n📝 EXTRACTED CODE:")
    print("-" * 40)
    print(row['extracted_code'][:500])
    if len(row['extracted_code']) > 500:
        print("...")


PROMPT COMPARISON (Sample Tasks)

Task 0 (HumanEval/0) [FAIL]

📝 ORIGINAL HUMANEVAL PROMPT:
----------------------------------------
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


📝 MBPP-STYLE PROMPT:
----------------------------------------
Write a function called `has_close_elements` that check if in given list of numbers, are any two numbers closer to each other than
    given threshold.

Function signature: def has_close_elements(numbers: List[float], threshold: float) -> bool

assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == Tru

In [48]:
# Show all failed tasks (non-comment-only)
print("\n" + "=" * 80)
print("FAILED TASKS (Generated Code but Tests Failed)")
print("=" * 80)

failed_code_df = df[(df['passed'] == False) & (df['is_comment_only'] == False)]

if len(failed_code_df) == 0:
    print("\n✅ No code failures (all passed or were comment-only)!")
else:
    print(f"\n{len(failed_code_df)} tasks generated code but failed tests:")
    
    # Show summary table
    display(failed_code_df[['task_id', 'task_idx', 'entry_point']].head(20))
    
    # Show first few failures in detail
    for idx, row in failed_code_df.head(3).iterrows():
        print(f"\n--- Task {row['task_idx']} ({row['task_id']}) ---")
        print(f"Entry point: {row['entry_point']}")
        print(f"\nExtracted code:")
        print(row['extracted_code'][:800])
        if len(row['extracted_code']) > 800:
            print("...")
        print(f"\nTest list (first 3):")
        for test in row['test_list'][:3]:
            print(f"  {test}")


FAILED TASKS (Generated Code but Tests Failed)

133 tasks generated code but failed tests:


,task_id,task_idx,entry_point
0,HumanEval/0,0,has_close_elements
1,HumanEval/1,1,separate_paren_groups
2,HumanEval/2,2,truncate_number
3,HumanEval/3,3,below_zero
5,HumanEval/5,5,intersperse
6,HumanEval/6,6,parse_nested_parens
8,HumanEval/8,8,sum_product
9,HumanEval/9,9,rolling_max
10,HumanEval/10,10,make_palindrome
11,HumanEval/11,11,string_xor



--- Task 0 (HumanEval/0) ---
Entry point: has_close_elements

Extracted code:
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for i in range(len(numbers) - 1):
        if abs(numbers[i] - numbers[i + 1]) < threshold:
            return True
    return False

Test list (first 3):
  assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
  assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
  assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True

--- Task 1 (HumanEval/1) ---
Entry point: separate_paren_groups

Extracted code:
def separate_paren_groups(paren_string: str) -> List[str]:
    paren_string = paren_string.replace(' ', '')
    paren_string = paren_string.replace('(', '')
    paren_string = paren_string.replace(')', '')
    paren_string = paren_string.replace('()', '')
    paren_string = paren_string.replace('(()', '')
    paren_string = paren_string.replace('(()', '')
    paren_string = paren_string.re

In [49]:
# Show all passed tasks
print("\n" + "=" * 80)
print("PASSED TASKS")
print("=" * 80)

passed_df = df[df['passed'] == True]

print(f"\n✅ {len(passed_df)} tasks passed:")
display(passed_df[['task_id', 'task_idx', 'entry_point']].head(20))


PASSED TASKS

✅ 31 tasks passed:


,task_id,task_idx,entry_point
4,HumanEval/4,4,mean_absolute_deviation
7,HumanEval/7,7,filter_by_substring
12,HumanEval/12,12,longest
13,HumanEval/13,13,greatest_common_divisor
15,HumanEval/15,15,string_sequence
18,HumanEval/18,18,how_many_times
23,HumanEval/23,23,strlen
25,HumanEval/25,25,factorize
26,HumanEval/26,26,remove_duplicates
28,HumanEval/28,28,concatenate


In [50]:
# Display all records
print(f"\n{'=' * 80}")
print(f"ALL RECORDS:")
print("=" * 80)
display(df)


ALL RECORDS:


,task_id,task_idx,passed,is_comment_only,original_prompt,mbpp_style_prompt,generated_text,extracted_code,test_list,entry_point
0,HumanEval/0,0,False,False,"from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n """""" Check if in given list of numbers, are any two numbers closer to each other than\n given threshold.\n >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n False\n >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n True\n """"""\n","Write a function called `has_close_elements` that check if in given list of numbers, are any two numbers closer to each other than\n given threshold.\n\nFunction signature: def has_close_elements(numbers: List[float], threshold: float) -> bool\n\nassert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\nassert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\nassert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\nassert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\nassert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n\n# Solution:","Write a function called `has_close_elements` that check if in given list of numbers, are any two numbers closer to each other than\n given threshold.\n\nFunction signature: def has_close_elements(numbers: List[float], threshold: float) -> bool\n\nassert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\nassert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\nassert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\nassert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\nassert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n\n# Solution:\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n for i in range(len(numbers) - 1):\n if abs(numbers[i] - numbers[i + 1]) < threshold:\n return True\n return False\n","def has_close_elements(numbers: List[float], threshold: float) -> bool:\n for i in range(len(numbers) - 1):\n if abs(numbers[i] - numbers[i + 1]) < threshold:\n return True\n return False","[assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True, assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False, assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True, assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False, assert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True, assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True, assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False]",has_close_elements
1,HumanEval/1,1,False,False,"from typing import List\n\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n """""" Input to this function is a string containing multiple groups of nested parentheses. Your goal is to\n separate those group into separate strings and return the list of those.\n Separate groups are balanced (each open brace is properly closed) and not nested within each other\n Ignore any spaces in the input string.\n >>> separate_paren_groups('( ) (( )) (( )( ))')\n ['()', '(())', '(()())']\n """"""\n","Write a function called `separate_paren_groups` that input to this function is a string containing multiple groups of nested parentheses. your goal is to\n separate those group into separate strings and return the list of those.\n separate groups are balanced (each open brace is properly closed) and not nested within each other\n ignore any spaces in the input string.\n\nFunction signature: def separate_paren_groups(paren_string: str) -> List[str]\n\nassert separate_paren_groups('(()()) ((())) () ((())()())') == [\nassert separate_paren_groups('() (()) ((())) (((())))') == [\nassert separate_paren_groups('(()(())((())))') == [\nassert separate_paren_groups('( ) (( )) (( )( ))') == ['()', '(())', '(()())']\n\n# Solution:","Write a function called `separate_paren_groups` that input to this function is a string containing multipl

In [ ]:
# Compact status table (quick overview)
print("\n" + "=" * 80)
print("COMPACT STATUS TABLE")
print("=" * 80)

compact_df = df[['task_id', 'task_idx', 'passed', 'is_comment_only', 'entry_point']].copy()
compact_df['status'] = compact_df.apply(
    lambda r: 'PASS' if r['passed'] else ('COMMENT' if r['is_comment_only'] else 'FAIL'), 
    axis=1
)
display(compact_df)

In [ ]:
# Summary
print("\n" + "=" * 80)
print("OVERALL SUMMARY")
print("=" * 80)

print(f"\n📊 Total tasks tested: {len(df)}")
print(f"✅ Passed: {n_passed} ({100*n_passed/len(df):.1f}%)")
print(f"❌ Failed (code): {n_failed_code} ({100*n_failed_code/len(df):.1f}%)")
print(f"💬 Comment-only: {n_comment_only} ({100*n_comment_only/len(df):.1f}%)")

print(f"\n🎯 Key Question: Did MBPP-style format eliminate comment-only failures?")
if n_comment_only == 0:
    print(f"   ✅ YES! No comment-only generations with MBPP-style format.")
else:
    print(f"   ❌ NO - Still {n_comment_only} comment-only generations.")

print("\n🎉 Inspection complete!")